# Loading Dataset

In [12]:
from glob import glob
from keras.utils import np_utils
from sklearn.datasets import load_files
import numpy as np


def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), 133)
    return dog_files, dog_targets


names = [item[20:-1] for item in sorted(glob("images/bones/train/*/"))]
train_files, train_targets = load_dataset('images/bones/train')
valid_files, valid_targets = load_dataset('images/bones/valid')
test_files, test_targets = load_dataset('images/bones/test')


# Loading Tensors

In [13]:
from keras.preprocessing import image
from tqdm import tqdm

def path_to_tensor(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)


# Pre-processing Images

In [14]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255


 86%|████████▋ | 221/256 [00:01<00:00, 92.66it/s] 

OSError: cannot identify image file 'images/bones/train/001.Fracture/fracture_00026.jpg'

 86%|████████▋ | 221/256 [00:20<00:03, 11.04it/s]

# Creating CNN Model

In [8]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

model = Sequential()

model.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(133, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(133, activation='softmax'))

model.summary()

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 224, 224, 16)      208       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 112, 112, 16)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 112, 112, 32)      2080      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 56, 56, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 56, 56, 64)        8256      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 28, 28, 64)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 28, 28, 64)        0         
__________

# Training The Model

In [9]:
from keras.callbacks import ModelCheckpoint

epochs = 10

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_scratch.hdf5',
                               verbose=1, save_best_only=True)

model.fit(train_tensors, train_targets, validation_data=(valid_tensors, valid_targets),
          epochs=epochs, batch_size=20, callbacks=[checkpointer], verbose=1)


Train on 6680 samples, validate on 835 samples
Epoch 1/10
6680/6680 [==============================] - 233s - loss: 4.9181 - acc: 0.0103 - val_loss: 4.8277 - val_acc: 0.0240
Epoch 2/10
6680/6680 [==============================] - 235s - loss: 4.7157 - acc: 0.0232 - val_loss: 4.5698 - val_acc: 0.0359
Epoch 3/10
6680/6680 [==============================] - 246s - loss: 4.4566 - acc: 0.0425 - val_loss: 4.3604 - val_acc: 0.0479
Epoch 4/10
6680/6680 [==============================] - 247s - loss: 4.2274 - acc: 0.0650 - val_loss: 4.1853 - val_acc: 0.0743
Epoch 5/10
6680/6680 [==============================] - 249s - loss: 3.9781 - acc: 0.0874 - val_loss: 4.1361 - val_acc: 0.0743
Epoch 6/10
6680/6680 [==============================] - 249s - loss: 3.7537 - acc: 0.1195 - val_loss: 4.1667 - val_acc: 0.0790
Epoch 7/10
6680/6680 [==============================] - 249s - loss: 3.4991 - acc: 0.1633 - val_loss: 4.2236 - val_acc: 0.0946
Epoch 8/10
6680/6680 [==============================] - 249s - l

# Testing The Model

In [10]:
model.load_weights('saved_models/weights.best.from_scratch.hdf5')

dog_breed_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

test_accuracy = 100*np.sum(np.array(dog_breed_predictions)==np.argmax(test_targets, axis=1))/len(dog_breed_predictions)
print('Bones test accuracy: %.4f%%' % test_accuracy)


Bones test accuracy: 7.5359%
